In [1]:
# !pip install pandas matplotlib numpy gcsfs scikit-learn

In [1]:
!curl -O "https://storage.googleapis.com/jfyang-test-us-west2/NYPD_Complaint_Data_Current__Year_To_Date__20240408.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  195M  100  195M    0     0  37.4M      0  0:00:05  0:00:05 --:--:-- 43.5M


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df_original = pd.read_csv("NYPD_Complaint_Data_Current__Year_To_Date__20240408.csv")

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
df_original.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,269235387,42.0,BRONX,06/03/2023,01:00:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,105,FELONY,(null),ROBBERY,(null),PATROL BORO BRONX,366.0,"ROBBERY,BICYCLE",STREET,06/03/2023,(null),UNKNOWN,BLACK,M,NaN,25-44,BLACK HISPANIC,M,1011982.0,239520.0,40.824059,-73.899799,"(40.82405906, -73.89979939)",POINT (-73.89979939 40.82405906)
1,262102261,43.0,BRONX,01/18/2023,14:00:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,113,FELONY,(null),FORGERY,(null),PATROL BORO BRONX,725.0,"FORGERY,M.V. REGISTRATION",STREET,01/18/2023,(null),18-24,BLACK,M,NaN,UNKNOWN,UNKNOWN,E,1019635.0,243829.0,40.835859,-73.872125,"(40.83585867, -73.87212505)",POINT (-73.87212505 40.83585867)
2,267972239,43.0,BRONX,05/07/2023,00:15:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,106,FELONY,FRONT OF,FELONY ASSAULT,(null),PATROL BORO BRONX,105.0,STRANGULATION 1ST,STREET,05/09/2023,(null),25-44,BLACK,M,NaN,25-44,BLACK,F,1022051.0,242244.0,40.831500,-73.863400,"(40.8315, -73.8634)",POINT (-73.8634 40.8315)
3,263141402H1,NaN,BROOKLYN,02/06/2023,10:16:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BKLYN SOUTH,NaN,(null),MULTI DWELL - APT BUILD,02/06/2023,(null),18-24,BLACK,M,NaN,18-24,BLACK,M,NaN,NaN,NaN,NaN,NaN,NaN
4,269307857H1,NaN,BROOKLYN,06/04/2023,00:20:00,NaN,(null),COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BKLYN NORTH,NaN,(null),(null),06/04/2023,(null),25-44,BLACK,M,NaN,25-44,BLACK,M,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# columns interested in
INTERESTED_CATEGORICAL_COLS = ['BORO_NM', 'LAW_CAT_CD', 'OFNS_DESC', 'PATROL_BORO', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']

# remove any row where any interested column has infrequent value, occuring less than 100 times
df_cleaned = df_original[INTERESTED_CATEGORICAL_COLS]
for c in INTERESTED_CATEGORICAL_COLS:
    df_cleaned = df_cleaned[df_cleaned[c].isin(df_cleaned[c].value_counts()[lambda x: x > 100].index.tolist())]
    # also convert (null) to UNKNOWN
    df_cleaned[c] = df_cleaned[c].replace('(null)', 'UNKNOWN')
print(f'Reduced {len(df_original)} columns to {len(df_cleaned)}')

for c in INTERESTED_CATEGORICAL_COLS:
    print(f'Column: {c}; Cardinality: {len(pd.unique(df_cleaned[c]))}')
    print(df_cleaned[c].value_counts())
    print()

Reduced 555117 columns to 554024
Column: BORO_NM; Cardinality: 6
BORO_NM
BROOKLYN         154324
MANHATTAN        132016
QUEENS           122077
BRONX            119816
STATEN ISLAND     24772
UNKNOWN            1019
Name: count, dtype: int64

Column: LAW_CAT_CD; Cardinality: 3
LAW_CAT_CD
MISDEMEANOR    281981
FELONY         186233
VIOLATION       85810
Name: count, dtype: int64

Column: OFNS_DESC; Cardinality: 43
OFNS_DESC
PETIT LARCENY                           109995
HARRASSMENT 2                            83883
ASSAULT 3 & RELATED OFFENSES             58289
GRAND LARCENY                            50459
CRIMINAL MISCHIEF & RELATED OF           42288
FELONY ASSAULT                           27826
VEHICLE AND TRAFFIC LAWS                 22928
OFF. AGNST PUB ORD SENSBLTY &            17946
MISCELLANEOUS PENAL LAW                  17517
ROBBERY                                  16896
GRAND LARCENY OF MOTOR VEHICLE           15789
BURGLARY                                 13731
DANGEROU

In [6]:
df_cleaned.head()

,BORO_NM,LAW_CAT_CD,OFNS_DESC,PATROL_BORO,PREM_TYP_DESC,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,BRONX,FELONY,ROBBERY,PATROL BORO BRONX,STREET,UNKNOWN,BLACK,M,25-44,BLACK HISPANIC,M
1,BRONX,FELONY,FORGERY,PATROL BORO BRONX,STREET,18-24,BLACK,M,UNKNOWN,UNKNOWN,E
2,BRONX,FELONY,FELONY ASSAULT,PATROL BORO BRONX,STREET,25-44,BLACK,M,25-44,BLACK,F
4,BROOKLYN,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,PATROL BORO BKLYN NORTH,UNKNOWN,25-44,BLACK,M,25-44,BLACK,M
8,MANHATTAN,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,PATROL BORO MAN SOUTH,UNKNOWN,25-44,BLACK,M,25-44,BLACK,M


In [7]:
# create dictionary and list of baskets
dictionary = set()
baskets = []
for i, row in df_cleaned.iterrows():
    basket = set()
    for c in INTERESTED_CATEGORICAL_COLS:
        if row[c] != 'UNKNOWN':
            item = f'{c}/{row[c]}'
            dictionary.add(item)
            basket.add(item)
    baskets.append(basket)

In [8]:
for i in range(5): print(baskets[i])

{'VIC_AGE_GROUP/25-44', 'OFNS_DESC/ROBBERY', 'LAW_CAT_CD/FELONY', 'SUSP_SEX/M', 'VIC_RACE/BLACK HISPANIC', 'VIC_SEX/M', 'PATROL_BORO/PATROL BORO BRONX', 'BORO_NM/BRONX', 'SUSP_RACE/BLACK', 'PREM_TYP_DESC/STREET'}
{'SUSP_AGE_GROUP/18-24', 'VIC_SEX/E', 'LAW_CAT_CD/FELONY', 'SUSP_SEX/M', 'PATROL_BORO/PATROL BORO BRONX', 'BORO_NM/BRONX', 'SUSP_RACE/BLACK', 'OFNS_DESC/FORGERY', 'PREM_TYP_DESC/STREET'}
{'VIC_AGE_GROUP/25-44', 'LAW_CAT_CD/FELONY', 'SUSP_SEX/M', 'PATROL_BORO/PATROL BORO BRONX', 'BORO_NM/BRONX', 'OFNS_DESC/FELONY ASSAULT', 'SUSP_RACE/BLACK', 'SUSP_AGE_GROUP/25-44', 'VIC_RACE/BLACK', 'PREM_TYP_DESC/STREET', 'VIC_SEX/F'}
{'PATROL_BORO/PATROL BORO BKLYN NORTH', 'VIC_AGE_GROUP/25-44', 'LAW_CAT_CD/FELONY', 'OFNS_DESC/MURDER & NON-NEGL. MANSLAUGHTER', 'SUSP_SEX/M', 'VIC_SEX/M', 'BORO_NM/BROOKLYN', 'SUSP_RACE/BLACK', 'SUSP_AGE_GROUP/25-44', 'VIC_RACE/BLACK'}
{'VIC_AGE_GROUP/25-44', 'BORO_NM/MANHATTAN', 'LAW_CAT_CD/FELONY', 'OFNS_DESC/MURDER & NON-NEGL. MANSLAUGHTER', 'SUSP_SEX/M', 'PA

In [9]:
df_cleaned.to_csv('data.csv', index=False)

print("csv saved!")

csv saved!


### "encoded_data.csv" to test with `apriori` from `mlxtend.frequent_patterns`

In [5]:
INTERESTED_CATEGORICAL_COLS = ['BORO_NM', 'LAW_CAT_CD', 'OFNS_DESC', 'PATROL_BORO', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']

df_cleaned = df_original[INTERESTED_CATEGORICAL_COLS]

In [6]:
df_cleaned = df_original[INTERESTED_CATEGORICAL_COLS]
for c in INTERESTED_CATEGORICAL_COLS:
    df_cleaned = df_cleaned[df_cleaned[c].isin(df_cleaned[c].value_counts()[lambda x: x > 100].index.tolist())]
    # also convert (null) to UNKNOWN
    df_cleaned[c] = df_cleaned[c].replace('(null)', 'UNKNOWN')
print(f'Reduced {len(df_original)} columns to {len(df_cleaned)}')

for c in INTERESTED_CATEGORICAL_COLS:
    print(f'Column: {c}; Cardinality: {len(pd.unique(df_cleaned[c]))}')
    print(df_cleaned[c].value_counts())
    print()

Reduced 555117 columns to 554024
Column: BORO_NM; Cardinality: 6
BORO_NM
BROOKLYN         154324
MANHATTAN        132016
QUEENS           122077
BRONX            119816
STATEN ISLAND     24772
UNKNOWN            1019
Name: count, dtype: int64

Column: LAW_CAT_CD; Cardinality: 3
LAW_CAT_CD
MISDEMEANOR    281981
FELONY         186233
VIOLATION       85810
Name: count, dtype: int64

Column: OFNS_DESC; Cardinality: 43
OFNS_DESC
PETIT LARCENY                           109995
HARRASSMENT 2                            83883
ASSAULT 3 & RELATED OFFENSES             58289
GRAND LARCENY                            50459
CRIMINAL MISCHIEF & RELATED OF           42288
FELONY ASSAULT                           27826
VEHICLE AND TRAFFIC LAWS                 22928
OFF. AGNST PUB ORD SENSBLTY &            17946
MISCELLANEOUS PENAL LAW                  17517
ROBBERY                                  16896
GRAND LARCENY OF MOTOR VEHICLE           15789
BURGLARY                                 13731
DANGEROU

In [7]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 554024 entries, 0 to 555116
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   BORO_NM         554024 non-null  object
 1   LAW_CAT_CD      554024 non-null  object
 2   OFNS_DESC       554024 non-null  object
 3   PATROL_BORO     554024 non-null  object
 4   PREM_TYP_DESC   554024 non-null  object
 5   SUSP_AGE_GROUP  554024 non-null  object
 6   SUSP_RACE       554024 non-null  object
 7   SUSP_SEX        554024 non-null  object
 8   VIC_AGE_GROUP   554024 non-null  object
 9   VIC_RACE        554024 non-null  object
 10  VIC_SEX         554024 non-null  object
dtypes: object(11)
memory usage: 50.7+ MB


In [10]:
df_encoded = pd.get_dummies(df_cleaned, columns=[
    "BORO_NM", "LAW_CAT_CD", "OFNS_DESC",
    "PATROL_BORO", "PREM_TYP_DESC", "SUSP_AGE_GROUP",
    "SUSP_RACE", "SUSP_SEX", "VIC_AGE_GROUP", 
    "VIC_RACE", "VIC_SEX"
])

df_encoded.head()

,BORO_NM_BRONX,BORO_NM_BROOKLYN,BORO_NM_MANHATTAN,BORO_NM_QUEENS,BORO_NM_STATEN ISLAND,BORO_NM_UNKNOWN,LAW_CAT_CD_FELONY,LAW_CAT_CD_MISDEMEANOR,LAW_CAT_CD_VIOLATION,OFNS_DESC_ADMINISTRATIVE CODE,OFNS_DESC_AGRICULTURE & MRKTS LAW-UNCLASSIFIED,OFNS_DESC_ARSON,OFNS_DESC_ASSAULT 3 & RELATED OFFENSES,OFNS_DESC_BURGLAR'S TOOLS,OFNS_DESC_BURGLARY,OFNS_DESC_CANNABIS RELATED OFFENSES,OFNS_DESC_CRIMINAL MISCHIEF & RELATED OF,OFNS_DESC_CRIMINAL TRESPASS,OFNS_DESC_DANGEROUS DRUGS,OFNS_DESC_DANGEROUS WEAPONS,OFNS_DESC_FELONY ASSAULT,OFNS_DESC_FORGERY,OFNS_DESC_FRAUDS,OFNS_DESC_GAMBLING,OFNS_DESC_GRAND LARCENY,OFNS_DESC_GRAND LARCENY OF MOTOR VEHICLE,OFNS_DESC_HARRASSMENT 2,OFNS_DESC_INTOXICATED & IMPAIRED DRIVING,OFNS_DESC_JOSTLING,OFNS_DESC_KIDNAPPING & RELATED OFFENSES,OFNS_DESC_MISCELLANEOUS PENAL LAW,OFNS_DESC_MURDER & NON-NEGL. MANSLAUGHTER,OFNS_DESC_NYS LAWS-UNCLASSIFIED FELONY,OFNS_DESC_OFF. AGNST PUB ORD SENSBLTY &,OFNS_DESC_OFFENSES AGAINST PUBLIC ADMINI,OFNS_DESC_OFFENSES AGAINST PUBLIC SAFETY,OFNS_DESC_OFFENSES AGAINST THE PERSON,OFNS_DESC_OFFENSES INVOLVING FRAUD,OFNS_DESC_OTHER OFFENSES RELATED TO THEF,OFNS_DESC_OTHER STATE LAWS,OFNS_DESC_OTHER STATE LAWS (NON PENAL LA,OFNS_DESC_PETIT LARCENY,OFNS_DESC_PETIT LARCENY OF MOTOR VEHICLE,OFNS_DESC_POSSESSION OF STOLEN PROPERTY,OFNS_DESC_PROSTITUTION & RELATED OFFENSES,OFNS_DESC_RAPE,OFNS_DESC_ROBBERY,OFNS_DESC_SEX CRIMES,OFNS_DESC_THEFT OF SERVICES,OFNS_DESC_THEFT-FRAUD,OFNS_DESC_UNAUTHORIZED USE OF A VEHICLE,OFNS_DESC_VEHICLE AND TRAFFIC LAWS,PATROL_BORO_PATROL BORO BKLYN NORTH,PATROL_BORO_PATROL BORO BKLYN SOUTH,PATROL_BORO_PATROL BORO BRONX,PATROL_BORO_PATROL BORO MAN NORTH,PATROL_BORO_PATROL BORO MAN SOUTH,PATROL_BORO_PATROL BORO QUEENS NORTH,PATROL_BORO_PATROL BORO QUEENS SOUTH,PATROL_BORO_PATROL BORO STATEN ISLAND,PREM_TYP_DESC_ABANDONED BUILDING,PREM_TYP_DESC_AIRPORT TERMINAL,PREM_TYP_DESC_ATM,PREM_TYP_DESC_BANK,PREM_TYP_DESC_BAR/NIGHT CLUB,PREM_TYP_DESC_BEAUTY & NAIL SALON,PREM_TYP_DESC_BOOK/CARD,PREM_TYP_DESC_BRIDGE,PREM_TYP_DESC_BUS (NYC TRANSIT),PREM_TYP_DESC_BUS (OTHER),PREM_TYP_DESC_BUS STOP,PREM_TYP_DESC_BUS TERMINAL,PREM_TYP_DESC_CANDY STORE,PREM_TYP_DESC_CHAIN STORE,PREM_TYP_DESC_CHECK CASHING BUSINESS,PREM_TYP_DESC_CHURCH,PREM_TYP_DESC_CLOTHING/BOUTIQUE,PREM_TYP_DESC_COLLEGE/UNIVERSITY,PREM_TYP_DESC_COMMERCIAL BUILDING,PREM_TYP_DESC_CONSTRUCTION SITE,PREM_TYP_DESC_DAYCARE FACILITY,PREM_TYP_DESC_DEPARTMENT STORE,PREM_TYP_DESC_DOCTOR/DENTIST OFFICE,PREM_TYP_DESC_DRUG STORE,PREM_TYP_DESC_DRY CLEANER/LAUNDRY,PREM_TYP_DESC_FACTORY/WAREHOUSE,PREM_TYP_DESC_FAST FOOD,PREM_TYP_DESC_FERRY/FERRY TERMINAL,PREM_TYP_DESC_FOOD SUPERMARKET,PREM_TYP_DESC_GAS STATION,PREM_TYP_DESC_GROCERY/BODEGA,PREM_TYP_DESC_GYM/FITNESS FACILITY,PREM_TYP_DESC_HIGHWAY/PARKWAY,PREM_TYP_DESC_HOMELESS SHELTER,PREM_TYP_DESC_HOSPITAL,PREM_TYP_DESC_HOTEL/MOTEL,PREM_TYP_DESC_JEWELRY,PREM_TYP_DESC_LIQUOR STORE,PREM_TYP_DESC_MAILBOX INSIDE,PREM_TYP_DESC_MAILBOX OUTSIDE,PREM_TYP_DESC_MARINA/PIER,PREM_TYP_DESC_MOBILE FOOD,PREM_TYP_DESC_OPEN AREAS (OPEN LOTS),PREM_TYP_DESC_OTHER,PREM_TYP_DESC_PARK/PLAYGROUND,PREM_TYP_DESC_PARKING LOT/GARAGE (PRIVATE),PREM_TYP_DESC_PARKING LOT/GARAGE (PUBLIC),PREM_TYP_DESC_PRIVATE/PAROCHIAL SCHOOL,PREM_TYP_DESC_PUBLIC BUILDING,PREM_TYP_DESC_PUBLIC SCHOOL,PREM_TYP_DESC_REAL ESTATE,PREM_TYP_DESC_RESIDENCE - APT. HOUSE,PREM_TYP_DESC_RESIDENCE - PUBLIC HOUSING,PREM_TYP_DESC_RESIDENCE-HOUSE,PREM_TYP_DESC_RESTAURANT/DINER,PREM_TYP_DESC_SHOE,PREM_TYP_DESC_SMALL MERCHANT,PREM_TYP_DESC_SMOKE SHOP,PREM_TYP_DESC_SOCIAL CLUB/POLICY,PREM_TYP_DESC_STORAGE FACILITY,PREM_TYP_DESC_STORE UNCLASSIFIED,PREM_TYP_DESC_STREET,PREM_TYP_DESC_SYNAGOGUE,PREM_TYP_DESC_TAXI (LIVERY LICENSED),PREM_TYP_DESC_TAXI (YELLOW LICENSED),PREM_TYP_DESC_TELECOMM. STORE,PREM_TYP_DESC_TRANSIT - NYC SUBWAY,PREM_TYP_DESC_TRANSIT FACILITY (OTHER),PREM_TYP_DESC_TUNNEL,PREM_TYP_DESC_UNKNOWN,PREM_TYP_DESC_VARIETY STORE,SUSP_AGE_GROUP_18-24,SUSP_AGE_GROUP_25-44,SUSP_AGE_GROUP_45-64,SUSP_AGE_GROUP_65+,SUSP_AGE_GROUP_<18,SUSP_AGE

In [11]:
df_encoded.to_csv('encoded_data.csv', index=False)

print("csv saved!")

csv saved!


In [84]:
df_original.describe()

,ADDR_PCT_CD,HOUSING_PSA,JURISDICTION_CODE,KY_CD,PD_CD,TRANSIT_DISTRICT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
count,555047.000000,36025.000000,555117.000000,555117.000000,554731.000000,14733.000000,5.551050e+05,555105.000000,555105.000000,555105.000000
mean,63.954051,6405.863428,0.528698,299.576752,418.028987,15.974004,1.005504e+06,207741.793875,40.736684,-73.922998
std,35.106302,13256.637379,5.396617,159.024032,227.691726,12.881583,2.165866e+04,29949.894252,0.112854,0.160446
min,1.000000,218.000000,0.000000,101.000000,100.000000,1.000000,0.000000e+00,0.000000,0.000000,-74.254377
25%,40.000000,472.000000,0.000000,115.000000,258.000000,3.000000,9.916100e+05,185478.000000,40.675719,-73.973423
50%,63.000000,696.000000,0.000000,341.000000,357.000000,12.000000,1.004938e+06,207065.000000,40.734955,-73.925311
75%,102.000000,1233.000000,0.000000,351.000000,638.000000,32.000000,1.017907e+06,235044.000000,40.811802,-73.878510
max,123.000000,72770.000000,97.000000,881.000000,969.000000,34.000000,1.067226e+06,271819.000000,40.912714,0.000000


In [85]:
df_original.isnull().sum()

CMPLNT_NUM                       0
ADDR_PCT_CD                     70
BORO_NM                          0
CMPLNT_FR_DT                     0
CMPLNT_FR_TM                     0
CMPLNT_TO_DT                 34738
CMPLNT_TO_TM                     0
CRM_ATPT_CPTD_CD                 0
HADEVELOPT                       0
HOUSING_PSA                 519092
JURISDICTION_CODE                0
JURIS_DESC                       0
KY_CD                            0
LAW_CAT_CD                       0
LOC_OF_OCCUR_DESC                0
OFNS_DESC                        0
PARKS_NM                         0
PATROL_BORO                      0
PD_CD                          386
PD_DESC                          0
PREM_TYP_DESC                    0
RPT_DT                           0
STATION_NAME                     0
SUSP_AGE_GROUP                   0
SUSP_RACE                        0
SUSP_SEX                         0
TRANSIT_DISTRICT            540384
VIC_AGE_GROUP                    0
VIC_RACE            

In [86]:
num_rows = df_original.shape[0]
print("Total number of rows: ", num_rows)

Total number of rows:  555117


In [87]:
# Create a copy of the original df
df = df_original.copy()

## Handle Missing Values

### Replace strings like "(NaN)" or "(null)" with `np.nan`

In [88]:
# Identify columns with any remaning null values.
# Check for strings like "(NaN)" or "(null)" that weren't caught by the `isnull()` method

In [89]:
def replace_with_nan(value):
    if value in ['(NaN)', '(null)']:
        return np.nan
    return value

In [90]:
df = df.applymap(replace_with_nan)

/var/folders/rf/py3knbjd6s371nfl3rnt855400tcqc/T/ipykernel_73031/4030091112.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(replace_with_nan)


### Drop columns with >90% missing values

In [91]:
missing_values_after = df.isnull().sum()

missing_percentage_after = (missing_values_after / len(df)) * 100

missing_summary_after = pd.DataFrame({'missing_values': missing_values_after, 'missing_percentage': missing_percentage_after})
print(missing_summary_after[missing_summary_after['missing_values'] > 0].sort_values(by='missing_percentage', ascending=False))

                          missing_values  missing_percentage
HADEVELOPT                        553360           99.683490
PARKS_NM                          551791           99.400847
STATION_NAME                      540384           97.345965
TRANSIT_DISTRICT                  540384           97.345965
HOUSING_PSA                       519092           93.510377
LOC_OF_OCCUR_DESC                 111769           20.134314
SUSP_AGE_GROUP                     72130           12.993657
SUSP_RACE                          72130           12.993657
SUSP_SEX                           72130           12.993657
CMPLNT_TO_DT                       34738            6.257780
CMPLNT_TO_TM                       34325            6.183381
PREM_TYP_DESC                      22212            4.001319
BORO_NM                             1019            0.183565
PD_CD                                386            0.069535
PD_DESC                              386            0.069535
VIC_RACE                

In [92]:
# Drop `HADEVELOPT`, `PARKS_NM`, and `STATION_NAME` columns, as they also have >90% missing values.

df.drop(['HADEVELOPT', 'PARKS_NM', 'STATION_NAME', 'TRANSIT_DISTRICT', 'HOUSING_PSA'], axis=1, inplace=True)

### Drop rows with high number of missing values

In [93]:
df['missing_values_count'] = df.isnull().sum(axis=1)

rows_with_missing_values = df[df['missing_values_count'] > 0]
print("Rows with missing values and their counts:\n", rows_with_missing_values[['missing_values_count']])

Rows with missing values and their counts:
         missing_values_count
0                          3
1                          3
2                          2
3                         11
4                         12
...                      ...
555109                     1
555110                     3
555114                     1
555115                     1
555116                     3

[201422 rows x 1 columns]


In [94]:
missing_value_counts_summary = rows_with_missing_values['missing_values_count'].value_counts().sort_index()
print("Summary of missing value counts per row:")
print(missing_value_counts_summary)

Summary of missing value counts per row:
missing_values_count
1     95581
2     26266
3     56123
4     16612
5      3543
6      2372
7       848
8        52
9        13
11        4
12        6
14        1
15        1
Name: count, dtype: int64


Based on the above summary, it seems reasonable to drop rows with a high number of missing values (i.e. 5+ missing values), especially given that there are 36 columns in total. The total number of rows that will be dropped is 6840 (6840/555117 = 0.0123 of the total number of rows), and this is small compared to the overall dataset, so this should not have significant impact. 

In [95]:
df = df[df['missing_values_count'] <= 4]

missing_value_counts_summary = df.isnull().sum(axis=1).value_counts().sort_index()
print("Summary of missing value counts per row after dropping rows with more than 4 missing values:")
print(missing_value_counts_summary)

Summary of missing value counts per row after dropping rows with more than 4 missing values:
0    353695
1     95581
2     26266
3     56123
4     16612
Name: count, dtype: int64


In [96]:
print(f"Number of rows after dropping these rows: {len(df)}")

Number of rows after dropping these rows: 548277


In [97]:
df.drop('missing_values_count', axis=1, inplace=True)

In [98]:
# Also drop `Lat_Lon` and `New Georeferenced Column` columns as they are redundant.
# i.e. contain the same information as `Latitude` and `Longitude` in different formats.

df.drop(['Lat_Lon', 'New Georeferenced Column'], axis=1, inplace=True)

df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,JURIS_DESC,...,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,269235387,42.0,BRONX,06/03/2023,01:00:00,NaN,NaN,COMPLETED,0,N.Y. POLICE DEPT,...,UNKNOWN,BLACK,M,25-44,BLACK HISPANIC,M,1011982.0,239520.0,40.824059,-73.899799
1,262102261,43.0,BRONX,01/18/2023,14:00:00,NaN,NaN,COMPLETED,0,N.Y. POLICE DEPT,...,18-24,BLACK,M,UNKNOWN,UNKNOWN,E,1019635.0,243829.0,40.835859,-73.872125
2,267972239,43.0,BRONX,05/07/2023,00:15:00,NaN,NaN,COMPLETED,0,N.Y. POLICE DEPT,...,25-44,BLACK,M,25-44,BLACK,F,1022051.0,242244.0,40.831500,-73.863400
9,278972931,123.0,STATEN ISLAND,12/14/2023,02:50:00,12/14/2023,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,...,25-44,WHITE,M,UNKNOWN,UNKNOWN,E,934105.0,138444.0,40.546531,-74.180430
10,278470767,121.0,STATEN ISLAND,12/04/2023,15:53:00,12/04/2023,16:00:00,COMPLETED,0,N.Y. POLICE DEPT,...,UNKNOWN,BLACK,M,25-44,BLACK,F,945652.0,167646.0,40.626742,-74.139049


### Impute Missing Values

In [99]:
number_of_rows_with_missing_values = df.isnull().any(axis=1).sum()

print("Number of rows with missing values:", number_of_rows_with_missing_values)

Number of rows with missing values: 194582


In [100]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

print("Columns with missing values:", columns_with_missing_values)

Columns with missing values: ['BORO_NM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PD_CD', 'PD_DESC', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX']


In [101]:
df[columns_with_missing_values].info()

<class 'pandas.core.frame.DataFrame'>
Index: 548277 entries, 0 to 555116
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   BORO_NM            548201 non-null  object 
 1   CMPLNT_TO_DT       519422 non-null  object 
 2   CMPLNT_TO_TM       519830 non-null  object 
 3   LOC_OF_OCCUR_DESC  440489 non-null  object 
 4   OFNS_DESC          548260 non-null  object 
 5   PD_CD              548135 non-null  float64
 6   PD_DESC            548135 non-null  object 
 7   PREM_TYP_DESC      527134 non-null  object 
 8   SUSP_AGE_GROUP     482837 non-null  object 
 9   SUSP_RACE          482837 non-null  object 
 10  SUSP_SEX           482837 non-null  object 
dtypes: float64(1), object(10)
memory usage: 50.2+ MB


In [102]:
for column in columns_with_missing_values:
    df[column].fillna('UNKNOWN', inplace=True)

/var/folders/rf/py3knbjd6s371nfl3rnt855400tcqc/T/ipykernel_73031/4167810133.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna('UNKNOWN', inplace=True)
/var/folders/rf/py3knbjd6s371nfl3rnt855400tcqc/T/ipykernel_73031/4167810133.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'UNKNOWN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[column].fillna('UNKNOWN', inplace=True)


In [103]:
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,JURIS_DESC,...,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,269235387,42.0,BRONX,06/03/2023,01:00:00,UNKNOWN,UNKNOWN,COMPLETED,0,N.Y. POLICE DEPT,...,UNKNOWN,BLACK,M,25-44,BLACK HISPANIC,M,1011982.0,239520.0,40.824059,-73.899799
1,262102261,43.0,BRONX,01/18/2023,14:00:00,UNKNOWN,UNKNOWN,COMPLETED,0,N.Y. POLICE DEPT,...,18-24,BLACK,M,UNKNOWN,UNKNOWN,E,1019635.0,243829.0,40.835859,-73.872125
2,267972239,43.0,BRONX,05/07/2023,00:15:00,UNKNOWN,UNKNOWN,COMPLETED,0,N.Y. POLICE DEPT,...,25-44,BLACK,M,25-44,BLACK,F,1022051.0,242244.0,40.831500,-73.863400
9,278972931,123.0,STATEN ISLAND,12/14/2023,02:50:00,12/14/2023,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,...,25-44,WHITE,M,UNKNOWN,UNKNOWN,E,934105.0,138444.0,40.546531,-74.180430
10,278470767,121.0,STATEN ISLAND,12/04/2023,15:53:00,12/04/2023,16:00:00,COMPLETED,0,N.Y. POLICE DEPT,...,UNKNOWN,BLACK,M,25-44,BLACK,F,945652.0,167646.0,40.626742,-74.139049


### Drop Irrelevant Columns

In [104]:
# Drop `CMPLNT_NUM` column, as it is randomly generated persistent ID for each complaint.
df.drop('CMPLNT_NUM', axis=1, inplace=True)

In [105]:
number_of_rows_with_missing_values = df.isnull().any(axis=1).sum()

print("Number of rows with missing values:", number_of_rows_with_missing_values)

Number of rows with missing values: 0


In [106]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

print("Columns with missing values:", columns_with_missing_values)

Columns with missing values: []


## Encode Categorical Variables

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 548277 entries, 0 to 555116
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ADDR_PCT_CD        548277 non-null  float64
 1   BORO_NM            548277 non-null  object 
 2   CMPLNT_FR_DT       548277 non-null  object 
 3   CMPLNT_FR_TM       548277 non-null  object 
 4   CMPLNT_TO_DT       548277 non-null  object 
 5   CMPLNT_TO_TM       548277 non-null  object 
 6   CRM_ATPT_CPTD_CD   548277 non-null  object 
 7   JURISDICTION_CODE  548277 non-null  int64  
 8   JURIS_DESC         548277 non-null  object 
 9   KY_CD              548277 non-null  int64  
 10  LAW_CAT_CD         548277 non-null  object 
 11  LOC_OF_OCCUR_DESC  548277 non-null  object 
 12  OFNS_DESC          548277 non-null  object 
 13  PATROL_BORO        548277 non-null  object 
 14  PD_CD              548277 non-null  object 
 15  PD_DESC            548277 non-null  object 
 16  PREM_TY

### Encoding Time & Date Based Columns 

In [108]:
df['CMPLNT_FR_DT'] = pd.to_datetime(df['CMPLNT_FR_DT'], errors='coerce')
df['CMPLNT_TO_DT'] = pd.to_datetime(df['CMPLNT_TO_DT'], errors='coerce')
df['CMPLNT_FR_TM'] = pd.to_datetime(df['CMPLNT_FR_TM'], format='%H:%M:%S', errors='coerce').dt.time
df['CMPLNT_TO_TM'] = pd.to_datetime(df['CMPLNT_TO_TM'], format='%H:%M:%S', errors='coerce').dt.time

/var/folders/rf/py3knbjd6s371nfl3rnt855400tcqc/T/ipykernel_73031/2892128746.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CMPLNT_TO_DT'] = pd.to_datetime(df['CMPLNT_TO_DT'], errors='coerce')


In [109]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

print("Columns with missing values:", columns_with_missing_values)

number_of_rows_with_missing_values = df.isnull().any(axis=1).sum()

print("Number of rows with missing values:", number_of_rows_with_missing_values)

Columns with missing values: ['CMPLNT_FR_DT', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM']
Number of rows with missing values: 29080


In [110]:
# Use backward fill to impute
for column in columns_with_missing_values:
    df[column] = df[column].bfill()

updated_number_of_rows_with_missing_values = df.isnull().any(axis=1).sum()
print("Updated number of rows with missing values:", updated_number_of_rows_with_missing_values)

Updated number of rows with missing values: 0


In [111]:
df.head()

,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,JURIS_DESC,KY_CD,...,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,42.0,BRONX,2023-06-03,01:00:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,105,...,UNKNOWN,BLACK,M,25-44,BLACK HISPANIC,M,1011982.0,239520.0,40.824059,-73.899799
1,43.0,BRONX,2023-01-18,14:00:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,113,...,18-24,BLACK,M,UNKNOWN,UNKNOWN,E,1019635.0,243829.0,40.835859,-73.872125
2,43.0,BRONX,2023-05-07,00:15:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,106,...,25-44,BLACK,M,25-44,BLACK,F,1022051.0,242244.0,40.831500,-73.863400
9,123.0,STATEN ISLAND,2023-12-14,02:50:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,235,...,25-44,WHITE,M,UNKNOWN,UNKNOWN,E,934105.0,138444.0,40.546531,-74.180430
10,121.0,STATEN ISLAND,2023-12-04,15:53:00,2023-12-04,16:00:00,COMPLETED,0,N.Y. POLICE DEPT,344,...,UNKNOWN,BLACK,M,25-44,BLACK,F,945652.0,167646.0,40.626742,-74.139049


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 548277 entries, 0 to 555116
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ADDR_PCT_CD        548277 non-null  float64       
 1   BORO_NM            548277 non-null  object        
 2   CMPLNT_FR_DT       548277 non-null  datetime64[ns]
 3   CMPLNT_FR_TM       548277 non-null  object        
 4   CMPLNT_TO_DT       548277 non-null  datetime64[ns]
 5   CMPLNT_TO_TM       548277 non-null  object        
 6   CRM_ATPT_CPTD_CD   548277 non-null  object        
 7   JURISDICTION_CODE  548277 non-null  int64         
 8   JURIS_DESC         548277 non-null  object        
 9   KY_CD              548277 non-null  int64         
 10  LAW_CAT_CD         548277 non-null  object        
 11  LOC_OF_OCCUR_DESC  548277 non-null  object        
 12  OFNS_DESC          548277 non-null  object        
 13  PATROL_BORO        548277 non-null  object       

### Handle anomalous values in `SUSP_AGE_GROUP` and `VIC_AGE_GROUP` columns before encoding

In [113]:
# First, get their unique values
unique_susp_age_groups = df['SUSP_AGE_GROUP'].unique()
print("Unique SUSP_AGE_GROUP values:")
print(unique_susp_age_groups)

unique_vic_age_groups = df['VIC_AGE_GROUP'].unique()
print("\nUnique VIC_AGE_GROUP values:")
print(unique_vic_age_groups)

Unique SUSP_AGE_GROUP values:
['UNKNOWN' '18-24' '25-44' '45-64' '<18' '65+' '929' '-62' '2023' '-956'
 '-953' '-6' '1022' '953' '925' '1023' '-951' '-963' '938' '-2' '967'
 '964' '942' '-47' '-976' '927' '-1' '-5' '933']

Unique VIC_AGE_GROUP values:
['25-44' 'UNKNOWN' '45-64' '65+' '<18' '18-24' '-970' '-6' '-60' '-30'
 '1022' '-964' '-58' '-61' '-968' '1023' '962' '-3' '1018' '-971' '-2'
 '-929' '953' '-46' '-64' '1020' '-953' '-975' '-962' '1013' '949' '-932'
 '-10' '-961' '-33']


In [114]:
valid_age_groups = {'<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN'}

In [115]:
# Handle anomalous values (e.g. -953, 1013)

# First, count anomalous values
anomalous_susp_age = df[~df['SUSP_AGE_GROUP'].isin(valid_age_groups)]
count_anomalous_susp_age = anomalous_susp_age.shape[0]

anomalous_vic_age = df[~df['VIC_AGE_GROUP'].isin(valid_age_groups)]
count_anomalous_vic_age = anomalous_vic_age.shape[0]

print(f"Anomalous SUSP_AGE_GROUP values: {count_anomalous_susp_age}")
print(f"Anomalous VIC_AGE_GROUP values: {count_anomalous_vic_age}")

Anomalous SUSP_AGE_GROUP values: 78
Anomalous VIC_AGE_GROUP values: 40


In [116]:
# Replace anomalous values with 'UNKNOWN'
df['SUSP_AGE_GROUP'] = df['SUSP_AGE_GROUP'].apply(lambda x: x if x in valid_age_groups else 'UNKNOWN')
df['VIC_AGE_GROUP'] = df['VIC_AGE_GROUP'].apply(lambda x: x if x in valid_age_groups else 'UNKNOWN')

print("Unique SUSP_AGE_GROUP values after replacement:", df['SUSP_AGE_GROUP'].unique())
print("Unique VIC_AGE_GROUP values after replacement:", df['VIC_AGE_GROUP'].unique())

Unique SUSP_AGE_GROUP values after replacement: ['UNKNOWN' '18-24' '25-44' '45-64' '<18' '65+']
Unique VIC_AGE_GROUP values after replacement: ['25-44' 'UNKNOWN' '45-64' '65+' '<18' '18-24']


In [117]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

print("Columns with missing values:", columns_with_missing_values)

number_of_rows_with_missing_values = df.isnull().any(axis=1).sum()

print("Number of rows with missing values:", number_of_rows_with_missing_values)

Columns with missing values: []
Number of rows with missing values: 0


In [118]:
df.head()

,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,JURIS_DESC,KY_CD,...,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,42.0,BRONX,2023-06-03,01:00:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,105,...,UNKNOWN,BLACK,M,25-44,BLACK HISPANIC,M,1011982.0,239520.0,40.824059,-73.899799
1,43.0,BRONX,2023-01-18,14:00:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,113,...,18-24,BLACK,M,UNKNOWN,UNKNOWN,E,1019635.0,243829.0,40.835859,-73.872125
2,43.0,BRONX,2023-05-07,00:15:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,106,...,25-44,BLACK,M,25-44,BLACK,F,1022051.0,242244.0,40.831500,-73.863400
9,123.0,STATEN ISLAND,2023-12-14,02:50:00,2023-12-14,02:52:00,COMPLETED,0,N.Y. POLICE DEPT,235,...,25-44,WHITE,M,UNKNOWN,UNKNOWN,E,934105.0,138444.0,40.546531,-74.180430
10,121.0,STATEN ISLAND,2023-12-04,15:53:00,2023-12-04,16:00:00,COMPLETED,0,N.Y. POLICE DEPT,344,...,UNKNOWN,BLACK,M,25-44,BLACK,F,945652.0,167646.0,40.626742,-74.139049


### Encode Rest of the Categorical Variables

In [119]:
columns_left = ['BORO_NM', 'CRM_ATPT_CPTD_CD', 'JURIS_DESC', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 
                'OFNS_DESC', 'PATROL_BORO', 'PD_DESC', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_SEX', 
                'SUSP_RACE', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']

In [120]:
unique_values = {}
for column in columns_left:
    unique_values[column] = df[column].unique()

for column, values in unique_values.items():
    print(f"Unique values for column '{column}': {values}")

Unique values for column 'BORO_NM': ['BRONX' 'STATEN ISLAND' 'BROOKLYN' 'MANHATTAN' 'UNKNOWN' 'QUEENS']
Unique values for column 'CRM_ATPT_CPTD_CD': ['COMPLETED' 'ATTEMPTED']
Unique values for column 'JURIS_DESC': ['N.Y. POLICE DEPT' 'N.Y. HOUSING POLICE' 'OTHER' 'STATN IS RAPID TRANS'
 'MTA POLICE DEPT' 'N.Y. STATE POLICE' 'PORT AUTHORITY'
 'NEW YORK CITY SHERIFF OFFICE' 'U.S. PARK POLICE' 'METRO NORTH'
 'NYC PARKS' 'TRI-BORO BRDG TUNNL' 'DEPT OF CORRECTIONS'
 'DISTRICT ATTORNEY OFFICE' 'N.Y. TRANSIT POLICE'
 'NYS DEPT TAX AND FINANCE' 'LONG ISLAND RAILRD' 'AMTRACK'
 'N.Y.C. DEPT OF PROBATION' 'HEALTH & HOSP CORP'
 'N.Y.C. DEPT OF HOMELESS SERVICES' 'N.Y. STATE PARKS']
Unique values for column 'LAW_CAT_CD': ['FELONY' 'MISDEMEANOR' 'VIOLATION']
Unique values for column 'LOC_OF_OCCUR_DESC': ['UNKNOWN' 'FRONT OF' 'INSIDE' 'OPPOSITE OF' 'REAR OF' 'OUTSIDE']
Unique values for column 'OFNS_DESC': ['ROBBERY' 'FORGERY' 'FELONY ASSAULT' 'DANGEROUS DRUGS'
 'ASSAULT 3 & RELATED OFFENSES' 'VEHICL

First encode all columns except for `JURIS_DESC`, `OFNS_DESC`, `PD_DESC`, `PREM_TYP_DESC`, which have long and/or various values.

In [121]:
columns_to_exclude = ['JURIS_DESC', 'OFNS_DESC', 'PD_DESC', 'PREM_TYP_DESC']
columns_to_encode = [col for col in columns_left if col not in columns_to_exclude]

In [122]:
print(columns_to_encode)
for column in columns_to_encode:
    unique_values = df[column].unique()
    print(f"Unique values for column '{column}': {unique_values}")

['BORO_NM', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'PATROL_BORO', 'SUSP_AGE_GROUP', 'SUSP_SEX', 'SUSP_RACE', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']
Unique values for column 'BORO_NM': ['BRONX' 'STATEN ISLAND' 'BROOKLYN' 'MANHATTAN' 'UNKNOWN' 'QUEENS']
Unique values for column 'CRM_ATPT_CPTD_CD': ['COMPLETED' 'ATTEMPTED']
Unique values for column 'LAW_CAT_CD': ['FELONY' 'MISDEMEANOR' 'VIOLATION']
Unique values for column 'LOC_OF_OCCUR_DESC': ['UNKNOWN' 'FRONT OF' 'INSIDE' 'OPPOSITE OF' 'REAR OF' 'OUTSIDE']
Unique values for column 'PATROL_BORO': ['PATROL BORO BRONX' 'PATROL BORO STATEN ISLAND' 'PATROL BORO BKLYN SOUTH'
 'PATROL BORO MAN SOUTH' 'PATROL BORO BKLYN NORTH'
 'PATROL BORO QUEENS NORTH' 'PATROL BORO MAN NORTH'
 'PATROL BORO QUEENS SOUTH']
Unique values for column 'SUSP_AGE_GROUP': ['UNKNOWN' '18-24' '25-44' '45-64' '<18' '65+']
Unique values for column 'SUSP_SEX': ['M' 'F' 'U' 'UNKNOWN']
Unique values for column 'SUSP_RACE': ['BLACK' 'WHITE' 'WHITE HISPANIC' '

In [123]:
from sklearn.preprocessing import OneHotEncoder

print("Missing values before encoding:", df.isnull().sum().sum())

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_columns = encoder.fit_transform(df[columns_to_encode])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(columns_to_encode))

print("Missing values in encoded data:", encoded_df.isnull().sum().sum())

df.reset_index(drop=True, inplace=True)

encoded_df.reset_index(drop=True, inplace=True)

df = pd.concat([df.drop(columns=columns_to_encode, axis=1), encoded_df], axis=1)

print("Missing values after encoding:", df.isnull().sum().sum())

Missing values before encoding: 0
Missing values in encoded data: 0
Missing values after encoding: 0


In [124]:
df.head()

,ADDR_PCT_CD,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,JURISDICTION_CODE,JURIS_DESC,KY_CD,OFNS_DESC,PD_CD,...,VIC_RACE_BLACK,VIC_RACE_BLACK HISPANIC,VIC_RACE_UNKNOWN,VIC_RACE_WHITE,VIC_RACE_WHITE HISPANIC,VIC_SEX_D,VIC_SEX_E,VIC_SEX_F,VIC_SEX_L,VIC_SEX_M
0,42.0,2023-06-03,01:00:00,2023-12-14,02:52:00,0,N.Y. POLICE DEPT,105,ROBBERY,366.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,43.0,2023-01-18,14:00:00,2023-12-14,02:52:00,0,N.Y. POLICE DEPT,113,FORGERY,725.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,43.0,2023-05-07,00:15:00,2023-12-14,02:52:00,0,N.Y. POLICE DEPT,106,FELONY ASSAULT,105.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,123.0,2023-12-14,02:50:00,2023-12-14,02:52:00,0,N.Y. POLICE DEPT,235,DANGEROUS DRUGS,511.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,121.0,2023-12-04,15:53:00,2023-12-04,16:00:00,0,N.Y. POLICE DEPT,344,ASSAULT 3 & RELATED OFFENSES,113.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Encode `JURIS_DESC`

In [125]:
unique_values = df['JURIS_DESC'].unique()
print("Unique values for column 'JURIS_DESC':")
print(unique_values)

Unique values for column 'JURIS_DESC':
['N.Y. POLICE DEPT' 'N.Y. HOUSING POLICE' 'OTHER' 'STATN IS RAPID TRANS'
 'MTA POLICE DEPT' 'N.Y. STATE POLICE' 'PORT AUTHORITY'
 'NEW YORK CITY SHERIFF OFFICE' 'U.S. PARK POLICE' 'METRO NORTH'
 'NYC PARKS' 'TRI-BORO BRDG TUNNL' 'DEPT OF CORRECTIONS'
 'DISTRICT ATTORNEY OFFICE' 'N.Y. TRANSIT POLICE'
 'NYS DEPT TAX AND FINANCE' 'LONG ISLAND RAILRD' 'AMTRACK'
 'N.Y.C. DEPT OF PROBATION' 'HEALTH & HOSP CORP'
 'N.Y.C. DEPT OF HOMELESS SERVICES' 'N.Y. STATE PARKS']


In [126]:
frequency = df['JURIS_DESC'].value_counts()

print(frequency)

JURIS_DESC
N.Y. POLICE DEPT                    492625
N.Y. HOUSING POLICE                  35363
N.Y. TRANSIT POLICE                  14606
PORT AUTHORITY                        2252
OTHER                                  899
MTA POLICE DEPT                        596
DEPT OF CORRECTIONS                    540
N.Y. STATE POLICE                      236
NYC PARKS                              206
NEW YORK CITY SHERIFF OFFICE           185
DISTRICT ATTORNEY OFFICE               146
METRO NORTH                            142
HEALTH & HOSP CORP                     120
TRI-BORO BRDG TUNNL                    112
AMTRACK                                 84
N.Y.C. DEPT OF HOMELESS SERVICES        82
N.Y. STATE PARKS                        26
LONG ISLAND RAILRD                      21
U.S. PARK POLICE                        13
N.Y.C. DEPT OF PROBATION                13
NYS DEPT TAX AND FINANCE                 8
STATN IS RAPID TRANS                     2
Name: count, dtype: int64


In [127]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

juris_encoded = encoder.fit_transform(df[['JURIS_DESC']])

juris_encoded_df = pd.DataFrame(juris_encoded, columns=encoder.get_feature_names_out(['JURIS_DESC']))

df = pd.concat([df.drop('JURIS_DESC', axis=1), juris_encoded_df], axis=1)

In [128]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

print("Columns with missing values:", columns_with_missing_values)

number_of_rows_with_missing_values = df.isnull().any(axis=1).sum()

print("Number of rows with missing values:", number_of_rows_with_missing_values)

Columns with missing values: []
Number of rows with missing values: 0


### Encode `OFNS_DESC`

In [129]:
pd.set_option('display.max_rows', None)

frequency = df['OFNS_DESC'].value_counts()

print(frequency)

OFNS_DESC
PETIT LARCENY                           106644
HARRASSMENT 2                            83713
ASSAULT 3 & RELATED OFFENSES             58318
GRAND LARCENY                            50022
CRIMINAL MISCHIEF & RELATED OF           41722
FELONY ASSAULT                           27839
VEHICLE AND TRAFFIC LAWS                 22412
OFF. AGNST PUB ORD SENSBLTY &            17901
MISCELLANEOUS PENAL LAW                  17475
ROBBERY                                  16905
GRAND LARCENY OF MOTOR VEHICLE           15674
BURGLARY                                 13717
DANGEROUS DRUGS                          12863
SEX CRIMES                                9119
DANGEROUS WEAPONS                         8076
OFFENSES AGAINST PUBLIC ADMINI            7154
FORGERY                                   5571
THEFT-FRAUD                               4157
INTOXICATED & IMPAIRED DRIVING            3356
OFFENSES INVOLVING FRAUD                  3342
FRAUDS                                    2736
OTH

In [130]:
low_freq_categories = frequency[frequency < 500].index.tolist()

print("Values with low frequency (under 500):")
print(low_freq_categories)

Values with low frequency (under 500):
["BURGLAR'S TOOLS", 'PETIT LARCENY OF MOTOR VEHICLE', 'CANNABIS RELATED OFFENSES', 'THEFT OF SERVICES', 'GAMBLING', 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', 'OTHER STATE LAWS', 'OFFENSES AGAINST PUBLIC SAFETY', 'PROSTITUTION & RELATED OFFENSES', 'KIDNAPPING & RELATED OFFENSES', 'MURDER & NON-NEGL. MANSLAUGHTER', 'JOSTLING', 'ALCOHOLIC BEVERAGE CONTROL LAW', 'CHILD ABANDONMENT/NON SUPPORT', 'FRAUDULENT ACCOSTING', 'DISORDERLY CONDUCT', 'OFFENSES RELATED TO CHILDREN', 'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'ESCAPE 3', 'ENDAN WELFARE INCOMP', 'UNKNOWN', 'KIDNAPPING', 'NEW YORK CITY HEALTH CODE', 'UNLAWFUL POSS. WEAP. ON SCHOOL', 'INTOXICATED/IMPAIRED DRIVING', 'ANTICIPATORY OFFENSES', 'NYS LAWS-UNCLASSIFIED VIOLATION', 'FELONY SEX CRIMES', 'DISRUPTION OF A RELIGIOUS SERV', 'OTHER TRAFFIC INFRACTION', 'FORTUNE TELLING', 'HOMICIDE-NEGLIGENT-VEHICLE', 'LOITERING']


In [131]:
ofns_mappings = {
    "BURGLAR'S TOOLS": 'THEFT',
    'PROSTITUTION & RELATED OFFENSES': 'SEX CRIMES',
    'FELONY SEX CRIMES': 'SEX CRIMES',
    'KIDNAPPING & RELATED OFFENSES': 'KIDNAPPING',
    'CHILD ABANDONMENT/NON SUPPORT': 'OFFENSES RELATED TO CHILDREN',
    'UNLAWFUL POSS. WEAP. ON SCHOOL': 'DANGEROUS WEAPONS',
    'OFFENSES AGAINST PUBLIC SAFETY': 'PUBLIC SAFETY OFFENSES',
    'DISORDERLY CONDUCT': 'PUBLIC SAFETY OFFENSES',
    'ESCAPE 3': 'PUBLIC SAFETY OFFENSES',
    'ALCOHOLIC BEVERAGE CONTROL LAW': 'REGULATORY OFFENSES',
    'AGRICULTURE & MRKTS LAW-UNCLASSIFIED': 'REGULATORY OFFENSES',
    'OTHER STATE LAWS': 'REGULATORY OFFENSES',
    'NYS LAWS-UNCLASSIFIED VIOLATION': 'REGULATORY OFFENSES',
    'HOMICIDE-NEGLIGENT,UNCLASSIFIE': 'HOMICIDE-NEGLIGENT',
    'HOMICIDE-NEGLIGENT-VEHICLE': 'HOMICIDE-NEGLIGENT',
}

def map_low_freq_offenses(offense):
    if offense in low_freq_categories:
        return ofns_mappings.get(offense, 'OTHER')
    else:
        return offense
        
df['OFNS_DESC'] = df['OFNS_DESC'].apply(map_low_freq_offenses)

In [132]:
unique_values = df['OFNS_DESC'].unique()
print("Unique values for column 'OFNS_DESC':")
print(unique_values)

Unique values for column 'OFNS_DESC':
['ROBBERY' 'FORGERY' 'FELONY ASSAULT' 'DANGEROUS DRUGS'
 'ASSAULT 3 & RELATED OFFENSES' 'VEHICLE AND TRAFFIC LAWS' 'BURGLARY'
 'SEX CRIMES' 'PETIT LARCENY' 'OTHER OFFENSES RELATED TO THEF'
 'CRIMINAL MISCHIEF & RELATED OF' 'GRAND LARCENY OF MOTOR VEHICLE'
 'HARRASSMENT 2' 'FRAUDS' 'OFF. AGNST PUB ORD SENSBLTY &' 'GRAND LARCENY'
 'INTOXICATED & IMPAIRED DRIVING' 'REGULATORY OFFENSES'
 'MISCELLANEOUS PENAL LAW' 'DANGEROUS WEAPONS' 'THEFT-FRAUD'
 'UNAUTHORIZED USE OF A VEHICLE' 'OFFENSES AGAINST PUBLIC ADMINI' 'ARSON'
 'NYS LAWS-UNCLASSIFIED FELONY' 'CRIMINAL TRESPASS' 'RAPE'
 'OFFENSES INVOLVING FRAUD' 'OTHER STATE LAWS (NON PENAL LA'
 'OFFENSES AGAINST THE PERSON' 'PUBLIC SAFETY OFFENSES'
 'POSSESSION OF STOLEN PROPERTY' 'OTHER' 'KIDNAPPING'
 'ADMINISTRATIVE CODE' 'HOMICIDE-NEGLIGENT' 'THEFT'
 'OFFENSES RELATED TO CHILDREN']


In [133]:
pd.set_option('display.max_rows', None)

frequency = df['OFNS_DESC'].value_counts()

print(frequency)

OFNS_DESC
PETIT LARCENY                     106644
HARRASSMENT 2                      83713
ASSAULT 3 & RELATED OFFENSES       58318
GRAND LARCENY                      50022
CRIMINAL MISCHIEF & RELATED OF     41722
FELONY ASSAULT                     27839
VEHICLE AND TRAFFIC LAWS           22412
OFF. AGNST PUB ORD SENSBLTY &      17901
MISCELLANEOUS PENAL LAW            17475
ROBBERY                            16905
GRAND LARCENY OF MOTOR VEHICLE     15674
BURGLARY                           13717
DANGEROUS DRUGS                    12863
SEX CRIMES                          9269
DANGEROUS WEAPONS                   8088
OFFENSES AGAINST PUBLIC ADMINI      7154
FORGERY                             5571
THEFT-FRAUD                         4157
INTOXICATED & IMPAIRED DRIVING      3356
OFFENSES INVOLVING FRAUD            3342
FRAUDS                              2736
OTHER OFFENSES RELATED TO THEF      2517
POSSESSION OF STOLEN PROPERTY       2463
CRIMINAL TRESPASS                   2448
UNAUTH

In [134]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_data = encoder.fit_transform(df[['OFNS_DESC']])

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['OFNS_DESC']))

df = pd.concat([df.drop('OFNS_DESC', axis=1), encoded_df], axis=1)

In [135]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

print("Columns with missing values:", columns_with_missing_values)

number_of_rows_with_missing_values = df.isnull().any(axis=1).sum()

print("Number of rows with missing values:", number_of_rows_with_missing_values)

Columns with missing values: []
Number of rows with missing values: 0


### Encode `PD_DESC`?? => Drop this column for now

In [136]:
pd.set_option('display.max_rows', None)

frequency = df['PD_DESC'].value_counts()

print(frequency)

PD_DESC
HARASSMENT,SUBD 3,4,5                                                      64587
LARCENY,PETIT FROM STORE-SHOPL                                             50576
ASSAULT 3                                                                  44133
ASSAULT 2,1,UNCLASSIFIED                                                   20766
HARASSMENT,SUBD 1,CIVILIAN                                                 19126
AGGRAVATED HARASSMENT 2                                                    16491
TRAFFIC,UNCLASSIFIED MISDEMEAN                                             13421
CRIMINAL MISCHIEF,UNCLASSIFIED 4                                           12298
LARCENY,GRAND OF AUTO                                                      11334
CRIMINAL CONTEMPT 1                                                        10881
MENACING,UNCLASSIFIED                                                      10673
MISCHIEF, CRIMINAL 4, OF MOTOR                                              9848
LARCENY,PETIT FROM B

In [137]:
df.drop('PD_DESC', axis=1, inplace=True)

### Encode `PREM_TYP_DESC`?? => Drop this column for now

In [138]:
unique_values = df['PREM_TYP_DESC'].unique()
print("Unique values for column 'PREM_TYP_DESC':")
print(unique_values)

Unique values for column 'PREM_TYP_DESC':
['STREET' 'RESIDENCE-HOUSE' 'BAR/NIGHT CLUB' 'SOCIAL CLUB/POLICY'
 'UNKNOWN' 'HOMELESS SHELTER' 'OTHER' 'COMMERCIAL BUILDING'
 'BUS (NYC TRANSIT)' 'BUS (OTHER)' 'CHAIN STORE'
 'RESIDENCE - PUBLIC HOUSING' 'STORE UNCLASSIFIED'
 'PARKING LOT/GARAGE (PUBLIC)' 'DEPARTMENT STORE' 'RESIDENCE - APT. HOUSE'
 'GROCERY/BODEGA' 'PUBLIC SCHOOL' 'GYM/FITNESS FACILITY'
 'DOCTOR/DENTIST OFFICE' 'GAS STATION' 'PARK/PLAYGROUND'
 'RESTAURANT/DINER' 'HOSPITAL' 'ATM' 'CHURCH' 'BANK' 'FOOD SUPERMARKET'
 'FAST FOOD' 'PARKING LOT/GARAGE (PRIVATE)' 'CLOTHING/BOUTIQUE'
 'CHECK CASHING BUSINESS' 'HIGHWAY/PARKWAY' 'DRUG STORE' 'CANDY STORE'
 'STORAGE FACILITY' 'PUBLIC BUILDING' 'BOOK/CARD' 'HOTEL/MOTEL'
 'DRY CLEANER/LAUNDRY' 'LIQUOR STORE' 'FACTORY/WAREHOUSE' 'SMALL MERCHANT'
 'JEWELRY' 'PRIVATE/PAROCHIAL SCHOOL' 'OPEN AREAS (OPEN LOTS)'
 'TELECOMM. STORE' 'SHOE' 'COLLEGE/UNIVERSITY' 'BUS STOP'
 'TRANSIT FACILITY (OTHER)' 'TAXI (LIVERY LICENSED)' 'MAILBOX OUTSIDE'
 'BEA

In [139]:
pd.set_option('display.max_rows', None)

frequency = df['PREM_TYP_DESC'].value_counts()

print(frequency)

PREM_TYP_DESC
STREET                          161348
RESIDENCE - APT. HOUSE          114525
RESIDENCE-HOUSE                  46070
RESIDENCE - PUBLIC HOUSING       35510
CHAIN STORE                      27765
UNKNOWN                          21143
TRANSIT - NYC SUBWAY             13981
DEPARTMENT STORE                 13831
COMMERCIAL BUILDING              11238
DRUG STORE                       11007
OTHER                             8973
HOMELESS SHELTER                  8376
GROCERY/BODEGA                    7902
RESTAURANT/DINER                  5083
CLOTHING/BOUTIQUE                 5050
PARK/PLAYGROUND                   4544
BAR/NIGHT CLUB                    4139
PUBLIC SCHOOL                     3627
FAST FOOD                         2843
HOTEL/MOTEL                       2390
PARKING LOT/GARAGE (PUBLIC)       2376
HOSPITAL                          2236
FOOD SUPERMARKET                  2157
HIGHWAY/PARKWAY                   1906
BANK                              1839
PARKING LOT

In [140]:
df.drop('PREM_TYP_DESC', axis=1, inplace=True)

## Evaluate

In [141]:
df.head()

,ADDR_PCT_CD,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,JURISDICTION_CODE,KY_CD,PD_CD,RPT_DT,X_COORD_CD,...,OFNS_DESC_POSSESSION OF STOLEN PROPERTY,OFNS_DESC_PUBLIC SAFETY OFFENSES,OFNS_DESC_RAPE,OFNS_DESC_REGULATORY OFFENSES,OFNS_DESC_ROBBERY,OFNS_DESC_SEX CRIMES,OFNS_DESC_THEFT,OFNS_DESC_THEFT-FRAUD,OFNS_DESC_UNAUTHORIZED USE OF A VEHICLE,OFNS_DESC_VEHICLE AND TRAFFIC LAWS
0,42.0,2023-06-03,01:00:00,2023-12-14,02:52:00,0,105,366.0,06/03/2023,1011982.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,43.0,2023-01-18,14:00:00,2023-12-14,02:52:00,0,113,725.0,01/18/2023,1019635.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,43.0,2023-05-07,00:15:00,2023-12-14,02:52:00,0,106,105.0,05/09/2023,1022051.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,123.0,2023-12-14,02:50:00,2023-12-14,02:52:00,0,235,511.0,12/14/2023,934105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,121.0,2023-12-04,15:53:00,2023-12-04,16:00:00,0,344,113.0,12/04/2023,945652.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
print(df.columns.tolist())
print("Total number of columns:", len(df.columns))

['ADDR_PCT_CD', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'JURISDICTION_CODE', 'KY_CD', 'PD_CD', 'RPT_DT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude', 'BORO_NM_BRONX', 'BORO_NM_BROOKLYN', 'BORO_NM_MANHATTAN', 'BORO_NM_QUEENS', 'BORO_NM_STATEN ISLAND', 'BORO_NM_UNKNOWN', 'CRM_ATPT_CPTD_CD_ATTEMPTED', 'CRM_ATPT_CPTD_CD_COMPLETED', 'LAW_CAT_CD_FELONY', 'LAW_CAT_CD_MISDEMEANOR', 'LAW_CAT_CD_VIOLATION', 'LOC_OF_OCCUR_DESC_FRONT OF', 'LOC_OF_OCCUR_DESC_INSIDE', 'LOC_OF_OCCUR_DESC_OPPOSITE OF', 'LOC_OF_OCCUR_DESC_OUTSIDE', 'LOC_OF_OCCUR_DESC_REAR OF', 'LOC_OF_OCCUR_DESC_UNKNOWN', 'PATROL_BORO_PATROL BORO BKLYN NORTH', 'PATROL_BORO_PATROL BORO BKLYN SOUTH', 'PATROL_BORO_PATROL BORO BRONX', 'PATROL_BORO_PATROL BORO MAN NORTH', 'PATROL_BORO_PATROL BORO MAN SOUTH', 'PATROL_BORO_PATROL BORO QUEENS NORTH', 'PATROL_BORO_PATROL BORO QUEENS SOUTH', 'PATROL_BORO_PATROL BORO STATEN ISLAND', 'SUSP_AGE_GROUP_18-24', 'SUSP_AGE_GROUP_25-44', 'SUSP_AGE_GROUP_45-64', 'SUSP_AGE_G

In [143]:
total_rows = len(df)
print("Total number of rows:", total_rows)

Total number of rows: 548277


In [144]:
zero_counts = (df == 0).sum()

print(zero_counts)

ADDR_PCT_CD                                         0
CMPLNT_FR_DT                                        0
CMPLNT_FR_TM                                        0
CMPLNT_TO_DT                                        0
CMPLNT_TO_TM                                        0
JURISDICTION_CODE                              492625
KY_CD                                               0
PD_CD                                               0
RPT_DT                                              0
X_COORD_CD                                          2
Y_COORD_CD                                          2
Latitude                                            2
Longitude                                           2
BORO_NM_BRONX                                  430051
BORO_NM_BROOKLYN                               395517
BORO_NM_MANHATTAN                              416649
BORO_NM_QUEENS                                 427118
BORO_NM_STATEN ISLAND                          523849
BORO_NM_UNKNOWN             

<strong>Note</strong>: I think we could consider dropping columns that predominantly contain zeros, say >99%, or add mappings to combine values that appear infrequently with other similar values...

In [145]:
threshold = 0.99 * len(df)
columns_with_many_zeros = zero_counts[zero_counts > threshold].index.tolist()

print(f"Columns where more than 99% of the values are zeros: {columns_with_many_zeros}")

Columns where more than 99% of the values are zeros: ['BORO_NM_UNKNOWN', 'LOC_OF_OCCUR_DESC_OUTSIDE', 'SUSP_RACE_AMERICAN INDIAN/ALASKAN NATIVE', 'VIC_RACE_AMERICAN INDIAN/ALASKAN NATIVE', 'VIC_SEX_L', 'JURIS_DESC_AMTRACK', 'JURIS_DESC_DEPT OF CORRECTIONS', 'JURIS_DESC_DISTRICT ATTORNEY OFFICE', 'JURIS_DESC_HEALTH & HOSP CORP', 'JURIS_DESC_LONG ISLAND RAILRD', 'JURIS_DESC_METRO NORTH', 'JURIS_DESC_MTA POLICE DEPT', 'JURIS_DESC_N.Y. STATE PARKS', 'JURIS_DESC_N.Y. STATE POLICE', 'JURIS_DESC_N.Y.C. DEPT OF HOMELESS SERVICES', 'JURIS_DESC_N.Y.C. DEPT OF PROBATION', 'JURIS_DESC_NEW YORK CITY SHERIFF OFFICE', 'JURIS_DESC_NYC PARKS', 'JURIS_DESC_NYS DEPT TAX AND FINANCE', 'JURIS_DESC_OTHER', 'JURIS_DESC_PORT AUTHORITY', 'JURIS_DESC_STATN IS RAPID TRANS', 'JURIS_DESC_TRI-BORO BRDG TUNNL', 'JURIS_DESC_U.S. PARK POLICE', 'OFNS_DESC_ADMINISTRATIVE CODE', 'OFNS_DESC_ARSON', 'OFNS_DESC_CRIMINAL TRESPASS', 'OFNS_DESC_FRAUDS', 'OFNS_DESC_HOMICIDE-NEGLIGENT', 'OFNS_DESC_INTOXICATED & IMPAIRED DRIVING'

In [146]:
df.to_csv('data.csv', index=False)

print("df successfully saved!")

df successfully saved!


In [147]:
df.head()

,ADDR_PCT_CD,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,JURISDICTION_CODE,KY_CD,PD_CD,RPT_DT,X_COORD_CD,...,OFNS_DESC_POSSESSION OF STOLEN PROPERTY,OFNS_DESC_PUBLIC SAFETY OFFENSES,OFNS_DESC_RAPE,OFNS_DESC_REGULATORY OFFENSES,OFNS_DESC_ROBBERY,OFNS_DESC_SEX CRIMES,OFNS_DESC_THEFT,OFNS_DESC_THEFT-FRAUD,OFNS_DESC_UNAUTHORIZED USE OF A VEHICLE,OFNS_DESC_VEHICLE AND TRAFFIC LAWS
0,42.0,2023-06-03,01:00:00,2023-12-14,02:52:00,0,105,366.0,06/03/2023,1011982.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,43.0,2023-01-18,14:00:00,2023-12-14,02:52:00,0,113,725.0,01/18/2023,1019635.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,43.0,2023-05-07,00:15:00,2023-12-14,02:52:00,0,106,105.0,05/09/2023,1022051.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,123.0,2023-12-14,02:50:00,2023-12-14,02:52:00,0,235,511.0,12/14/2023,934105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,121.0,2023-12-04,15:53:00,2023-12-04,16:00:00,0,344,113.0,12/04/2023,945652.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [151]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
df.head(100)

,ADDR_PCT_CD,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,JURISDICTION_CODE,KY_CD,PD_CD,RPT_DT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,BORO_NM_BRONX,BORO_NM_BROOKLYN,BORO_NM_MANHATTAN,BORO_NM_QUEENS,BORO_NM_STATEN ISLAND,BORO_NM_UNKNOWN,CRM_ATPT_CPTD_CD_ATTEMPTED,CRM_ATPT_CPTD_CD_COMPLETED,LAW_CAT_CD_FELONY,LAW_CAT_CD_MISDEMEANOR,LAW_CAT_CD_VIOLATION,LOC_OF_OCCUR_DESC_FRONT OF,LOC_OF_OCCUR_DESC_INSIDE,LOC_OF_OCCUR_DESC_OPPOSITE OF,LOC_OF_OCCUR_DESC_OUTSIDE,LOC_OF_OCCUR_DESC_REAR OF,LOC_OF_OCCUR_DESC_UNKNOWN,PATROL_BORO_PATROL BORO BKLYN NORTH,PATROL_BORO_PATROL BORO BKLYN SOUTH,PATROL_BORO_PATROL BORO BRONX,PATROL_BORO_PATROL BORO MAN NORTH,PATROL_BORO_PATROL BORO MAN SOUTH,PATROL_BORO_PATROL BORO QUEENS NORTH,PATROL_BORO_PATROL BORO QUEENS SOUTH,PATROL_BORO_PATROL BORO STATEN ISLAND,SUSP_AGE_GROUP_18-24,SUSP_AGE_GROUP_25-44,SUSP_AGE_GROUP_45-64,SUSP_AGE_GROUP_65+,SUSP_AGE_GROUP_<18,SUSP_AGE_GROUP_UNKNOWN,SUSP_SEX_F,SUSP_SEX_M,SUSP_SEX_U,SUSP_SEX_UNKNOWN,SUSP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,SUSP_RACE_ASIAN / PACIFIC ISLANDER,SUSP_RACE_BLACK,SUSP_RACE_BLACK HISPANIC,SUSP_RACE_UNKNOWN,SUSP_RACE_WHITE,SUSP_RACE_WHITE HISPANIC,VIC_AGE_GROUP_18-24,VIC_AGE_GROUP_25-44,VIC_AGE_GROUP_45-64,VIC_AGE_GROUP_65+,VIC_AGE_GROUP_<18,VIC_AGE_GROUP_UNKNOWN,VIC_RACE_AMERICAN INDIAN/ALASKAN NATIVE,VIC_RACE_ASIAN / PACIFIC ISLANDER,VIC_RACE_BLACK,VIC_RACE_BLACK HISPANIC,VIC_RACE_UNKNOWN,VIC_RACE_WHITE,VIC_RACE_WHITE HISPANIC,VIC_SEX_D,VIC_SEX_E,VIC_SEX_F,VIC_SEX_L,VIC_SEX_M,JURIS_DESC_AMTRACK,JURIS_DESC_DEPT OF CORRECTIONS,JURIS_DESC_DISTRICT ATTORNEY OFFICE,JURIS_DESC_HEALTH & HOSP CORP,JURIS_DESC_LONG ISLAND RAILRD,JURIS_DESC_METRO NORTH,JURIS_DESC_MTA POLICE DEPT,JURIS_DESC_N.Y. HOUSING POLICE,JURIS_DESC_N.Y. POLICE DEPT,JURIS_DESC_N.Y. STATE PARKS,JURIS_DESC_N.Y. STATE POLICE,JURIS_DESC_N.Y. TRANSIT POLICE,JURIS_DESC_N.Y.C. DEPT OF HOMELESS SERVICES,JURIS_DESC_N.Y.C. DEPT OF PROBATION,JURIS_DESC_NEW YORK CITY SHERIFF OFFICE,JURIS_DESC_NYC PARKS,JURIS_DESC_NYS DEPT TAX AND FINANCE,JURIS_DESC_OTHER,JURIS_DESC_PORT AUTHORITY,JURIS_DESC_STATN IS RAPID TRANS,JURIS_DESC_TRI-BORO BRDG TUNNL,JURIS_DESC_U.S. PARK POLICE,OFNS_DESC_ADMINISTRATIVE CODE,OFNS_DESC_ARSON,OFNS_DESC_ASSAULT 3 & RELATED OFFENSES,OFNS_DESC_BURGLARY,OFNS_DESC_CRIMINAL MISCHIEF & RELATED OF,OFNS_DESC_CRIMINAL TRESPASS,OFNS_DESC_DANGEROUS DRUGS,OFNS_DESC_DANGEROUS WEAPONS,OFNS_DESC_FELONY ASSAULT,OFNS_DESC_FORGERY,OFNS_DESC_FRAUDS,OFNS_DESC_GRAND LARCENY,OFNS_DESC_GRAND LARCENY OF MOTOR VEHICLE,OFNS_DESC_HARRASSMENT 2,OFNS_DESC_HOMICIDE-NEGLIGENT,OFNS_DESC_INTOXICATED & IMPAIRED DRIVING,OFNS_DESC_KIDNAPPING,OFNS_DESC_MISCELLANEOUS PENAL LAW,OFNS_DESC_NYS LAWS-UNCLASSIFIED FELONY,OFNS_DESC_OFF. AGNST PUB ORD SENSBLTY &,OFNS_DESC_OFFENSES AGAINST PUBLIC ADMINI,OFNS_DESC_OFFENSES AGAINST THE PERSON,OFNS_DESC_OFFENSES INVOLVING FRAUD,OFNS_DESC_OFFENSES RELATED TO CHILDREN,OFNS_DESC_OTHER,OFNS_DESC_OTHER OFFENSES RELATED TO THEF,OFNS_DESC_OTHER STATE LAWS (NON PENAL LA,OFNS_DESC_PETIT LARCENY,OFNS_DESC_POSSESSION OF STOLEN PROPERTY,OFNS_DESC_PUBLIC SAFETY OFFENSES,OFNS_DESC_RAPE,OFNS_DESC_REGULATORY OFFENSES,OFNS_DESC_ROBBERY,OFNS_DESC_SEX CRIMES,OFNS_DESC_THEFT,OFNS_DESC_THEFT-FRAUD,OFNS_DESC_UNAUTHORIZED USE OF A VEHICLE,OFNS_DESC_VEHICLE AND TRAFFIC LAWS
0,42.0,2023-06-03,01:00:00,2023-12-14,02:52:00,0,105,366.0,06/03/2023,1011982.0,239520.0,40.824059,-73.899799,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,43.0,2023-01-18,14:00:00,2023-12-14,02:52:00,0,113,725.0,01/18/2023,1019635.0,243829.0,40.835859,-73.872125,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0